# SETUP

In [224]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import json

# IMPORT MAPPING + CREATE DF

In [225]:
#import the classification table
df_mapping = pd.read_excel('Classification_excel.xlsx', index_col=None)
df_mapping["Key words"] = df_mapping["Key words"].str.lower()

#Create df for each of the 4 categories
df_type = df_mapping.loc[df_mapping['Category'] == 'CLA type']
df_status = df_mapping.loc[df_mapping['Category'] == 'CLA status']
df_sector = df_mapping.loc[df_mapping['Category'] == 'Sector']
df_theme = df_mapping.loc[df_mapping['Category'] == 'Theme']

class_type_label_ind = df_type.loc[df_type['Key words'].isnull(), 'Class'].to_string(index=False)
class_type_label_ind

'Concluded on industry level'

# PIPEPLINE PDF --> CREATE LIST TEXT + CLASS

In [226]:
# GO FOR ALL FILE IN 200
# OUTCOME : [pdf unique id, cla type, status, sector, theme : ["salaires"],["durée de travail"]]

In [227]:
all_pdf_titles = [f for f in listdir("200") if isfile(join("200", f))]


In [228]:
dict_pdf = {}
for pdf in all_pdf_titles :
    pdf_class = {}
    pdf_txt = open(f"200/{pdf}", 'r')
    pdf_words = pdf_txt.read().lower()

    # Add the type to the list pdf_class
    class_type_label_cnt = df_type.loc[df_type['Key words'] == "www.cnt-nar.be", 'Class'].to_string(index=False)
    class_type_label_ind = df_type.loc[df_type['Key words'].isnull(), 'Class'].to_string(index=False)

    elements = df_type["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            if class_type_label_cnt not in pdf_class :
                pdf_class["CLA type"] = class_type_label_cnt
                break
        else : 
            if class_type_label_ind not in pdf_class :
                pdf_class["CLA type"] = class_type_label_ind

    # Add the status to the list pdf_class
    class_status_label_update = df_status.loc[df_status['Key words'] == "erratum", 'Class'].to_string(index=False)
    class_status_label_new = df_status.loc[df_status['Key words'].isnull(), 'Class'].to_string(index=False)

    elements = df_status["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            if class_status_label_update not in pdf_class :
                pdf_class["CLA status"] = class_status_label_update
                break
        else : 
            if class_status_label_new not in pdf_class :
                pdf_class["CLA status"] = class_status_label_new

    # Add the sector to the list pdf_class
    class_sector_label = df_sector['Class'].to_list()
    result_2 = []
    elements = df_sector["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            result_2.append(True)
        else : 
            result_2.append(False)

    if True in result_2 :
        pdf_class["Sector"] = df_sector.loc[df_sector["Key words"]== elem, "Class"].to_string(index=False)
    else :
        if "Sector not specified" not in pdf_class :
            pdf_class["Sector"] = "Sector not specified"

    #Add the theme to the list pdf_class

    class_theme_label = df_theme['Class'].to_list()
    result_3 = []
    elements = df_theme["Key words"].dropna().tolist()
    for elem in elements :
        if elem in pdf_words:
            result_3.append(True)
        else : 
            result_3.append(False)
    class_theme = []
    if True in result_3 :
        for elem in elements :
            if elem in pdf_words :
                if df_theme.loc[df_theme["Key words"]== elem, "Class"].to_string(index=False) not in pdf_class :
                    class_theme.append((df_theme.loc[df_theme["Key words"]== elem, "Class"].to_string(index=False)))
                    pdf_class["Theme"] = class_theme
    else :
        if "Unknown_theme" not in pdf_class :
            pdf_class["Theme"] =  "Unknown_theme"
    dict_pdf[pdf] = pdf_class
dict_pdf

#Changer Quel format vont les pdf arriver --> change "txt"

{'200-2021-013464_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'Sector not specified',
  'Theme': ['Fin au contrat de travail et chômage',
   'Primes et indemnités pour le travailleur actif']},
 '200-2021-013468_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'Sector not specified',
  'Theme': ['Salaire',
   'Temps de travail',
   'Recrutement et formation',
   "Mesures pour l'emploi et groupes cibles",
   'Fonds sociaux',
   "Fonctionnement au sein du comité/de l'entreprise",
   'Primes et indemnités pour le travailleur actif']},
 '200-2021-013476_fr.txt': {'CLA type': 'Concluded on industry level',
  'CLA status': 'New',
  'Sector': 'LA SELLERIE, DE LA FABRICATION DE COURROIES ET ...',
  'Theme': ["Mesures pour l'emploi et groupes cibles",
   'Fin au contrat de travail et chômage',
   'Primes et indemnités pour le travailleur actif']},
 '200-2021-013474_fr.txt': {'CLA type': 'Concluded on industry leve

# Merge with METADATA

## Create dataframe for csv meta and classification for pdfs

In [229]:
# Create DF for classification

df_dict_pdf = pd.DataFrame(dict_pdf).T.reset_index(level=0)
df_dict_pdf.rename(columns={df_dict_pdf.columns[0]: "filename"}, inplace = True)
df_dict_pdf["filename"] = df_dict_pdf["filename"].str.rstrip('_fr.txt')

In [230]:
# Create DF with meta
df_csv = pd.read_csv("CLA_meta_from_2018.csv")
df_csv["filename"] = df_csv["filename"].str.rstrip('.pdf')
df_csv = df_csv.loc[df_csv["filename"].str.startswith('200', na=False)]

In [231]:
# Merge both df on filename
merged_df = pd.merge(df_csv, df_dict_pdf, on="filename")
merged_df

,deposit_number,jc_number,jc_name_fr,jc_name_nl,cla_title_fr,cla_title_nl,retrieve_date,label,themes_fr,themes_nl,...,filename,doc_size,scope_fr,scope_nl,no_scope_fr,no_scope_nl,CLA type,CLA status,Sector,Theme
0,174187,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Pension complémentaire pour les employés de l'...,Aanvullend pensioen voor de bedienden van de s...,2022-12-07,new,pensions compémentaires et assurances groupes\...,aanvullende pensioenen en groepsverzekeringen\...,...,200-2022-009993,807 Kb,industrie des briques,steenbakkerij,NaN,NaN,Concluded on industry level,New,"LA SELLERIE, DE LA FABRICATION DE COURROIES ET...","[Salaire, Fin au contrat de travail et chômage..."
1,169162,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Prime corona,Coronapremie,2022-12-07,new,"chèques cadeau, sport et culture\prime propre ...","geschenken-, sport- en cultuurcheques\premie e...",...,200-2021-015135,84 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,Sector not specified,[Primes et indemnités pour le travailleur actif]
2,168835,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Dispense de l'obligation de disponibilité adap...,Vrijstelling van de verplichting van aangepast...,2022-12-07,new,NaN,NaN,...,200-2021-013476,249 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,"LA SELLERIE, DE LA FABRICATION DE COURROIES ET...","[Mesures pour l'emploi et groupes cibles, Fin ..."
3,168825,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Modification de la CCT concernant la prime de ...,Wijziging van de cao tot de eindejaarspremie,2022-12-07,new,prime de fin d'année\toutes les primes et inde...,eindejaarspremie\alle premies en vergoedingen ...,...,200-2021-013464,78 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,Sector not specified,"[Fin au contrat de travail et chômage, Primes ..."
4,168826,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Pouvoir d'achat,Koopkracht,2022-12-07,new,salaires\paix sociale\salaire horaire/mensuel ...,lonen\sociale vrede\minimumuur- en maandlonen\...,...,200-2021-013465,194 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,Sector not specified,"[Salaire, Salaire, Salaire, Salaire, Fin au co..."
5,168827,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Modification de la CCT relative au pouvoir d'a...,Wijziging van de cao betreffende de koopkracht,2022-12-07,modified,prime propre au secteur ou à l'entreprise\pens...,premie eigen aan de sector of aan de ondernemi...,...,200-2021-013466,177 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,Update,Sector not specified,[Fin au contrat de travail et chômage]
6,168828,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Modification de la CCT relative au FSE,Wijziging van de cao betreffende het FBZ,2022-12-07,new,fonds de sécurité d'existence\fonds de sécurit...,fondsen voor bestaanszekerheid\fondsen voor be...,...,200-2021-013467,99 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,Sector not specified,"[Salaire, Salaire, Mesures pour l'emploi et gr..."
7,168829,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Formation 2022-2023,Opleiding 2022-2023,2022-12-07,new,formation (excl. formation syndicale)\formatio...,opleiding (excl. syndicale vorming)\opleiding ...,...,200-2021-013468,522 Kb,NaN,NaN,NaN,NaN,Concluded on industry level,New,Sector not specified,"[Salaire, Temps de travail, Recrutement et for..."
8,168830,200.0,commission paritaire auxiliaire pour employes,aanvullend paritair comite voor de bedienden,Régime de chômage avec complément d'entreprise...,Stelsel van werkloosheid met bedrijfstoeslag -...,2022-12-07,new,régime de chômage avec complément d'entreprise...,stelsel va